# 0. Setup

Create an MLRun project and the streams relevant for our scenario.

![Model deployment with streaming Real-time operational Pipeline](../../assets/images/model-deployment-with-streaming.png)

### Create V3IO client

In [6]:
from os import getenv, path
from v3io import dataplane
from mlrun import new_project, mlconf


V3IO_ACCESS_KEY = getenv('V3IO_ACCESS_KEY')
V3IO_USERNAME = getenv('V3IO_USERNAME')
CONTAINER = 'users'

v3io_client = dataplane.Client(endpoint='http://v3io-webapi:8081', access_key=V3IO_ACCESS_KEY)


### Create MLRun Project

In [3]:
project_name = '-'.join(filter(None, ['rapid-prototype', getenv('V3IO_USERNAME', None)]))
project_path = path.abspath('conf')
project = new_project(project_name, project_path, init_git=True)

print(f'Project path: {project_path}\nProject name: {project_name}')


# Target location for storing pipeline artifacts
artifact_path = path.abspath('artifacts')
# MLRun DB path or API service URL
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'

print(f'Artifacts path: {artifact_path}\nMLRun DB path: {mlconf.dbpath}')

Project path: /User/rapid-proto/conf
Project name: rapid-prototype-michaelk
Artifacts path: /User/rapid-proto/artifacts
MLRun DB path: http://mlrun-api:8080


In [15]:
STREAM_CONFIGS = {'generated-stream': {
                        'path': path.join(V3IO_USERNAME, 'examples/rapid-prototype/generated-stream'),
                        'shard_count': 8},
                  'incoming-events-stream': {
                        'path': path.join(V3IO_USERNAME, 'examples/rapid-prototype/incoming-events-stream'),
                        'shard_count': 8
                  },
                  'enriched-events-stream': {
                        'path': path.join(V3IO_USERNAME, 'examples/rapid-prototype/enriched-events-stream'),
                        'shard_count': 8
                  },
                  'serving-stream': {
                        'path': path.join(V3IO_USERNAME, 'examples/rapid-prototype/serving-stream'),
                        'shard_count': 8
                  },
                  'inference-stream': {
                        'path': path.join(V3IO_USERNAME, 'examples/rapid-prototype/inference-stream'),
                        'shard_count': 8
                  }
                 }
config_path = path.join(artifact_path, 'config')
project.log_artifact('stream_config', artifact_path=config_path, body=json.dumps(STREAM_CONFIGS).encode('utf-8'), format='json')

In [16]:
project.to_dict()

{'name': 'rapid-prototype-michaelk',
 'functions': [],
 'workflows': [],
 'artifacts': [{'key': 'stream_config',
   'kind': '',
   'iter': 0,
   'tree': 'latest',
   'target_path': '/User/rapid-proto/artifacts/config/stream_config.json',
   'hash': '030bf2a9bcdf7b351ff43600b78efeb3456fddb3',
   'format': 'json',
   'size': 530,
   'db_key': 'stream_config'}],
 'artifact_path': ''}

## Managa Streams

#### Delete all streams

In [11]:
for stream_name, stream_config in STREAM_CONFIGS.items():
    resp = v3io_client.delete_stream(container=CONTAINER, path=stream_config['path'], 
                                     raise_for_status=dataplane.RaiseForStatus.never)
    print(f'Delete Stream call for stream {stream_name} returned with status {resp.status_code}, and content: {resp.body.decode("utf-8")}')
    

2020-08-03 14:15:57,665 [info] Remote disconnected while waiting for response: {'retries_left': 1, 'connection_idx': 5}
2020-08-03 14:15:57,665 [info] Remote disconnected while waiting for response: {'retries_left': 1, 'connection_idx': 5}
Delete Stream call for stream generated-stream returned with status 404, and content: {
	"ErrorCode": -2,
	"ErrorMessage": "No such file or directory"
}
2020-08-03 14:15:57,669 [info] Remote disconnected while waiting for response: {'retries_left': 1, 'connection_idx': 6}
2020-08-03 14:15:57,669 [info] Remote disconnected while waiting for response: {'retries_left': 1, 'connection_idx': 6}
2020-08-03 14:15:57,672 [info] Remote disconnected while waiting for response: {'retries_left': 1, 'connection_idx': 7}
2020-08-03 14:15:57,672 [info] Remote disconnected while waiting for response: {'retries_left': 1, 'connection_idx': 7}
2020-08-03 14:15:57,674 [info] Remote disconnected while waiting for response: {'retries_left': 1, 'connection_idx': 0}
2020-08

#### Create all streams

In [ ]:
for stream_name, stream_config in STREAM_CONFIGS.items():
    resp = v3io_client.create_stream(container=CONTAINER,
                                     path=stream_config['path'],
                                     shard_count=stream_config['shard_count'],
                                    raise_for_status=dataplane.RaiseForStatus.never)
    print(f'Create Stream call for stream {stream_name} returned with status {resp.status_code}, and content: {resp.body.decode("utf-8")}')
    